In [ ]:
import sys
sys.path.append('../')

In [ ]:
import os
import torch
from mei.methods import gradient_ascent
from mei.modules import ConstrainedOutputModel
import urllib
import datajoint as dj

import matplotlib.pyplot as plt
from nnfabrik.builder import get_model


In [ ]:
# only need to run once at the beginning
!pip3 install git+https://github.com/sinzlab/nnvision.git@model_builder

### Download monkey V4 example model files

In [ ]:
# download example model
url = 'https://github.com/sinzlab/nnvision/raw/model_builder/nnvision/data/model_weights/v4_task_driven/task_driven_ensemble_model_01.pth.tar'
filepath = './pretrained_models/task_driven_ensemble_model_01.pth.tar'
urllib.request.urlretrieve(url, filepath)

In [ ]:
# some hackity hack to supress datajoint promts (ignore)
dj.config["database.host"] = "{host_address}"
dj.config["database.user"] = "{user}"
dj.config["database.password"] = "{password}"

In [ ]:
# method for loading example model
def get_v4_model():
    model_fn = 'nnvision.models.ptrmodels.task_core_gauss_readout'
    model_config =  {'input_channels': 1,
      'model_name': 'resnet50_l2_eps0_1',
      'layer_name': 'layer3.0',
      'pretrained': False,
      'bias': False,
      'final_batchnorm': True,
      'final_nonlinearity': True,
      'momentum': 0.1,
      'fine_tune': False,
      'init_mu_range': 0.4,
      'init_sigma_range': 0.6,
      'readout_bias': True,
      'gamma_readout': 3.0,
      'gauss_type': 'isotropic',
      'elu_offset': -1,
                     }
    
    data_info = {
        "all_sessions": {
            "input_dimensions": torch.Size([64, 1, 100, 100]),
            "input_channels": 1,
            "output_dimension": 1244,
            "img_mean": 124.54466,
            "img_std": 70.28,
        },
    }
    
    filename = os.path.join('./pretrained_models/task_driven_ensemble_model_01.pth.tar')
    state_dict = torch.load(filename, map_location='cpu')
    
    v4_data_task_sota = get_model(
        model_fn, model_config, seed=10, data_info=data_info, state_dict=state_dict,
    )
    
    return v4_data_task_sota, data_info



# Load the model


In [ ]:
model, data_info = get_v4_model()

# Create single cell model
Here you select the index of the cell for which you'd like to optimize the MEI

In [ ]:
cell_index = 403

In [ ]:
single_cell_model = ConstrainedOutputModel(model, cell_index)

# Configurations
Configurations for the MEI optimization algorithm

In [ ]:
seed = 8

# learning rate
lr = 1

# number of iterations for which the MEI is optimized
num_iterations = 1000 

# interval with which the objective will be evaluated
interval = 10

# shape of the MEI
input_shape = (1,) + data_info["all_sessions"]['input_dimensions'][1:] 

device='cpu'

In [ ]:
method_config = dict(
    initial=dict(path="mei.initial.RandomNormal"),
    optimizer=dict(path="torch.optim.SGD", kwargs=dict(lr=lr)),
    stopper=dict(path="mei.stoppers.NumIterations", kwargs=dict(num_iterations=num_iterations)),
    objectives=[dict(path="mei.objectives.EvaluationObjective", kwargs=dict(interval=interval))],
    device=device,
)

# Optimize MEI

In [ ]:
mei, final_evaluation, tracker_log = gradient_ascent(model=single_cell_model, config=method_config, seed=seed, shape=input_shape)   

# Visualize MEI

In [ ]:
print(mei.shape)
print(torch.max(torch.abs(mei)))
plt.imshow(mei[0,0], cmap='gray')

In [ ]:
plt.plot(
    tracker_log["mei.objectives.EvaluationObjective"]["times"],
    tracker_log["mei.objectives.EvaluationObjective"]["values"],
)
plt.gca().set_xlabel("# iteration")
plt.gca().set_ylabel("evaluation")

# Constraints 
So far, we have not introduced contraints into MEI optimization. 
This results in implausible MEIs with high pixel values and high contrast and a forever increasing activation. We need to constrain the overall norm of the MEI and also clip the range of pixel values to the range of pixel values which were in the training data. 
The norm is then established empirically such, that ideally, value clipping is not necessary.

In [ ]:
p = 2
norm_value = 10
max_pixel_value = 2.2606
min_pixel_value = -1.75047

method_config = dict(
    initial=dict(path="mei.initial.RandomNormal"),
    optimizer=dict(path="torch.optim.SGD", kwargs=dict(lr=lr)),
    stopper=dict(path="mei.stoppers.NumIterations", kwargs=dict(num_iterations=num_iterations)),
    objectives=[dict(path="mei.objectives.EvaluationObjective", kwargs=dict(interval=interval))],
    postprocessing=dict(path="mei.postprocessing.PNormConstraintAndClip", kwargs=dict(p=p, norm_value=norm_value, max_pixel_value=max_pixel_value, min_pixel_value=min_pixel_value)),
    device=device,
)

In [ ]:
mei, final_evaluation, tracker_log = gradient_ascent(model=single_cell_model, config=method_config, seed=seed,
                                                     shape=input_shape)   

In [ ]:
print(mei.shape)
max_value = torch.max(torch.abs(mei))
print(max_value)
plt.imshow(mei[0,0], cmap='gray', vmin=min_pixel_value, vmax=max_pixel_value)
plt.colorbar()
plt.show()


In [ ]:
plt.plot(
    tracker_log["mei.objectives.EvaluationObjective"]["times"],
    tracker_log["mei.objectives.EvaluationObjective"]["values"],
)
plt.gca().set_xlabel("# iteration")
plt.gca().set_ylabel("evaluation")